# Data Cleaning and Data Wrangling

## Data

Dataset contains socio-demographic and firmographic features about 2.240 
customers.

|Feature |Description|
|--:|---|
|AcceptedCmp1| 1 if customer accepted the offer in the 1st campaign, 0 otherwise|
|AcceptedCmp2| 1 if customer accepted the offer in the 2nd campaign, 0 otherwise|
|AcceptedCmp3| 1 if customer accepted the offer in the 3rd campaign, 0 otherwise|
|AcceptedCmp4| 1 if customer accepted the offer in the 4th campaign, 0 otherwise|
|AcceptedCmp5| 1 if customer accepted the offer in the 5th campaign, 0 otherwise|
|Response (target)| 1 if customer accepted the offer in the last campaign, 0 otherwise|
|Complain| 1 if customer complained in the last 2 years|
|DtCustomer| date of customer's enrollment with the company|
|Education| customer's level of education|
|Marital| customer's marital status|
|Kidhome| number of small children in customer's household|
|Teenhome |number of teenagers in customer's household|
|Income| customer's yearly household income|
|MntFishProducts| amount spent on fish products in the last 2 years|
|MntMeatProducts| amount spent on meat products in the last 2 years|
|MntFruits| amount spent on fruits products in the last 2 years|
|MntSweetProducts| amount spent on sweet products in the last 2 years|
|MntWines| amount spent on wines products in the last 2 years|
|MntGoldProds| amount spent on gold products in the last 2 years|
|NumDealsPurchases| number of purchases made with discount|
|NunCatalogPurchases| number of purchases made using catalog|
|NunStorePurchases| number of purchases made directly in stores|
|NumWebPurchases| number of purchases made through company's web site|
|NumWebVisitsMonth| number of visits to company's web site in the last month|
|Recency|number of days since the last purchase|
|Z_Revenue|revenue from the new gadget|
|Z_CostContact|cost of contact for the sixth campaign|

In [1]:
import numpy as np
import pandas as pd
import json
from pathlib import Path
from datetime import datetime
from scipy import stats
from scipy.stats import kstest

import psutil
import warnings
warnings.filterwarnings("ignore")

## Data Exploration

In [2]:
# Storing path
path = Path("../data/ifood_customers.csv")

# Read CSV with pandas
data = pd.read_csv(path)

# showing a sample of the dataset
data.sample(8)

,ID,Year_Birth,Education,Marital_Status,Income,Kidhome,Teenhome,Dt_Customer,Recency,MntWines,...,NumWebVisitsMonth,AcceptedCmp3,AcceptedCmp4,AcceptedCmp5,AcceptedCmp1,AcceptedCmp2,Complain,Z_CostContact,Z_Revenue,Response
935,6383,1971,PhD,Together,32892.0,1,0,2014-05-25,78,26,...,8,0,0,0,0,0,0,3,11,0
1216,232,1965,Graduation,Single,61559.0,0,1,2013-07-17,8,279,...,3,0,0,0,0,0,0,3,11,0
660,55,1963,Graduation,Together,56253.0,0,1,2012-12-07,83,509,...,6,0,0,0,0,0,0,3,11,0
1976,5117,1963,2n Cycle,Married,32632.0,0,0,2012-08-02,32,63,...,5,0,0,0,0,0,0,3,11,0
781,10839,1976,Graduation,Together,36283.0,1,1,2014-04-14,42,6,...,4,0,0,0,0,0,0,3,11,0
1785,9648,1967,Graduation,Single,46904.0,1,1,2012-11-03,20,153,...,8,0,0,0,0,0,0,3,11,0
71,10629,1973,2n Cycle,Married,NaN,1,0,2012-09-14,25,25,...,8,0,0,0,0,0,0,3,11,0
412,7030,1955,PhD,Married,66465.0,0,1,2013-03-30,1,1200,...,6,0,0,0,1,0,0,3,11,0


In [3]:
# Exploring the columns and dtype

data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2240 entries, 0 to 2239
Data columns (total 29 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   ID                   2240 non-null   int64  
 1   Year_Birth           2240 non-null   int64  
 2   Education            2240 non-null   object 
 3   Marital_Status       2240 non-null   object 
 4   Income               2216 non-null   float64
 5   Kidhome              2240 non-null   int64  
 6   Teenhome             2240 non-null   int64  
 7   Dt_Customer          2240 non-null   object 
 8   Recency              2240 non-null   int64  
 9   MntWines             2240 non-null   int64  
 10  MntFruits            2240 non-null   int64  
 11  MntMeatProducts      2240 non-null   int64  
 12  MntFishProducts      2240 non-null   int64  
 13  MntSweetProducts     2240 non-null   int64  
 14  MntGoldProds         2240 non-null   int64  
 15  NumDealsPurchases    2240 non-null   i

In [4]:
# Exploring unique values in some columns

print("\n'Education' Column Values:\n", "\t",
      data["Education"].unique(),
      sep=''
      )

print("\n'Marital_Status' Column Values:\n", "\t",
      data["Marital_Status"].unique(),
      sep=''
      )

print("\n'Dt_Customer' Column sample:\n",
      data["Dt_Customer"].sample(3), "\n", "\t",
      sep=''
      )



'Education' Column Values:
	['Graduation' 'PhD' 'Master' 'Basic' '2n Cycle']

'Marital_Status' Column Values:
	['Single' 'Together' 'Married' 'Divorced' 'Widow' 'Alone' 'Absurd' 'YOLO']

'Dt_Customer' Column sample:
2115    2013-12-04
1235    2014-05-26
1191    2012-11-17
Name: Dt_Customer, dtype: object
	


## Creating columns

In [5]:
# Stabilizing dtype 'category' to Education and Marital_Status columns
# Also Dt_Customer to Datetime dtype and Income to 'Int64'

data["Income"] = data["Income"].astype("Int64")
data["Education"] = data["Education"].astype("category")
data["Marital_Status"] = data["Marital_Status"].astype("category")
data["Dt_Customer"] = pd.to_datetime(data["Dt_Customer"], format="%Y-%m-%d")

data.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2240 entries, 0 to 2239
Data columns (total 29 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   ID                   2240 non-null   int64         
 1   Year_Birth           2240 non-null   int64         
 2   Education            2240 non-null   category      
 3   Marital_Status       2240 non-null   category      
 4   Income               2216 non-null   Int64         
 5   Kidhome              2240 non-null   int64         
 6   Teenhome             2240 non-null   int64         
 7   Dt_Customer          2240 non-null   datetime64[ns]
 8   Recency              2240 non-null   int64         
 9   MntWines             2240 non-null   int64         
 10  MntFruits            2240 non-null   int64         
 11  MntMeatProducts      2240 non-null   int64         
 12  MntFishProducts      2240 non-null   int64         
 13  MntSweetProducts     2240 non-nul

In [6]:
# Creating new columns: customer year old and days since becoming customer

today = pd.to_datetime(datetime.today().strftime('%Y-%m-%d'))

data["Year_Old"] = (today.year - data["Year_Birth"])
data["CustomerFor"] = (today - data["Dt_Customer"])

data[["Year_Old", "CustomerFor"]].sample(5)

,Year_Old,CustomerFor
1756,56,3902 days
1385,76,3832 days
932,73,4353 days
1576,42,3765 days
55,69,4254 days


In [7]:
# Reordering columns

pop_column = data.pop("Dt_Customer")
data.insert(2, "Dt_Customer", pop_column)

last_columns = data.columns[-2:]
first_columns = data.columns[:2]
middle_columns = data.columns[2:-2]
new_order = list(first_columns) + list(last_columns) + list(middle_columns)

data = data[new_order]

data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2240 entries, 0 to 2239
Data columns (total 31 columns):
 #   Column               Non-Null Count  Dtype          
---  ------               --------------  -----          
 0   ID                   2240 non-null   int64          
 1   Year_Birth           2240 non-null   int64          
 2   Year_Old             2240 non-null   int64          
 3   CustomerFor          2240 non-null   timedelta64[ns]
 4   Dt_Customer          2240 non-null   datetime64[ns] 
 5   Education            2240 non-null   category       
 6   Marital_Status       2240 non-null   category       
 7   Income               2216 non-null   Int64          
 8   Kidhome              2240 non-null   int64          
 9   Teenhome             2240 non-null   int64          
 10  Recency              2240 non-null   int64          
 11  MntWines             2240 non-null   int64          
 12  MntFruits            2240 non-null   int64          
 13  MntMeatProducts   

In [8]:
# Checking if the customer bought in the last month

data["PurchaseLastMonth"] = (data["Recency"] < 30)
data["PurchaseLastMonth"] = data["PurchaseLastMonth"].replace({True:1, 
                                                               False:0})

print(data["PurchaseLastMonth"].sample(5))

1151    0
2113    0
79      1
456     0
195     1
Name: PurchaseLastMonth, dtype: int64


In [9]:
# Calculating total amount spent per customer

MntSpentTotal_sum = ["MntFishProducts", "MntFruits", "MntGoldProds", 
                     "MntMeatProducts", "MntSweetProducts", "MntWines"]
data["MntSpentTotal"] = data[MntSpentTotal_sum].sum(axis=1)

print(data["MntSpentTotal"].sample(5))

1304    1305
873      182
84       778
2135     395
574       62
Name: MntSpentTotal, dtype: int64


In [10]:
# How many campaigns the customer accepted

AcceptedCmpTotal_sum = ["AcceptedCmp1", "AcceptedCmp2", "AcceptedCmp3", 
                        "AcceptedCmp4", "AcceptedCmp5"]
data["AcceptedCmpTotal"] = data[AcceptedCmpTotal_sum].sum(axis=1)

data["AcceptedCmpTotal"].sample(5)

1996    1
2090    0
1767    0
131     0
1261    1
Name: AcceptedCmpTotal, dtype: int64

In [11]:
# How many children (Kids and teenagers) the customer has at home

ChildrenHome_sum = ["Kidhome", "Teenhome"]
data["ChildrenHome"] = data[ChildrenHome_sum].sum(axis="columns")

data["ChildrenHome"].sample(5)

1347    0
986     0
890     1
1715    1
452     1
Name: ChildrenHome, dtype: int64

### Columns reordered

In [12]:
# Reordering columns

pop_column = data.pop("AcceptedCmpTotal")
data.insert(27, "AcceptedCmpTotal", pop_column)

pop_column = data.pop("PurchaseLastMonth")
data.insert(17, "PurchaseLastMonth", pop_column)

pop_column = data.pop("MntSpentTotal")
data.insert(11, "MntSpentTotal", pop_column)

pop_column = data.pop("ChildrenHome")
data.insert(10, "ChildrenHome", pop_column)

pop_column = data.pop("AcceptedCmp2")
data.insert(25, "AcceptedCmp2", pop_column)

pop_column = data.pop("AcceptedCmp1")
data.insert(25, "AcceptedCmp1", pop_column)

pop_column = data.pop("Response")
data.insert(30, "Response", pop_column)

data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2240 entries, 0 to 2239
Data columns (total 35 columns):
 #   Column               Non-Null Count  Dtype          
---  ------               --------------  -----          
 0   ID                   2240 non-null   int64          
 1   Year_Birth           2240 non-null   int64          
 2   Year_Old             2240 non-null   int64          
 3   CustomerFor          2240 non-null   timedelta64[ns]
 4   Dt_Customer          2240 non-null   datetime64[ns] 
 5   Education            2240 non-null   category       
 6   Marital_Status       2240 non-null   category       
 7   Income               2216 non-null   Int64          
 8   Kidhome              2240 non-null   int64          
 9   Teenhome             2240 non-null   int64          
 10  ChildrenHome         2240 non-null   int64          
 11  Recency              2240 non-null   int64          
 12  MntSpentTotal        2240 non-null   int64          
 13  MntWines          

## Data Cleaning

In [13]:
# Missing Values

row_nan = data[data.isna().any(axis=1)]
print("Customers with missing values: ", len(row_nan))

data.drop(row_nan.index, inplace=True)

Customers with missing values:  24


In [14]:
# Identifying logically incoherent customers and dropping from the dataframe

marital_filt = data[data["Marital_Status"].isin(['Alone', 'Absurd', 'YOLO'])]
print("Customers with a logically incoherent Marital Status: ", 
      len(marital_filt))

data.drop(marital_filt.index, inplace=True)

Customers with a logically incoherent Marital Status:  7


### Kolmogorov-Smirnov test

In [15]:
# KS-Test on 'Year_Old' column
ks_result = kstest(data["Year_Old"], stats.norm.cdf, 
                   args=(data["Year_Old"].mean(), data["Year_Old"].std()))

print(f"Test statistic: {ks_result.statistic:.4f}")
print(f"p Value: {ks_result.pvalue:.4f}")

Test statistic: 0.0590
p Value: 0.0000


In [16]:
# KS-Test on 'Year_Old' column
ks_result = kstest(data["Income"], stats.norm.cdf, 
                   args=(data["Income"].mean(), data["Income"].std()))

print(f"Test statistic: {ks_result.statistic:.4f}")
print(f"p Value: {ks_result.pvalue:.4f}")

Test statistic: 0.0542
p Value: 0.0000


### Obtaining outliers

In [17]:
# Quartiles and IQR
quartiles = data["Year_Old"].quantile([0.25, 0.75])
iqr = quartiles[0.75] - quartiles[0.25]

# Identify Outliers
lower_bound = quartiles[0.25] - 1.5 * iqr
upper_bound = quartiles[0.75] + 1.5 * iqr

# Filtering
Year_Old_outliers = data[(data["Year_Old"] < lower_bound) | 
                (data["Year_Old"] > upper_bound)]

print("IQR: ", iqr)
print("outliers: ")
print(Year_Old_outliers[["ID", "Year_Old", "Income", "CustomerFor", 
                "Marital_Status", "MntSpentTotal"]])

# Extracting Outliers from the Dataset
data.drop(Year_Old_outliers.index, inplace=True)

IQR:  18.0
outliers: 
        ID  Year_Old  Income CustomerFor Marital_Status  MntSpentTotal
192   7829       124   36640   3943 days       Divorced             65
239  11004       131   60182   3710 days         Single             22
339   1150       125   83532   3943 days       Together           1853


In [18]:
# Quartiles and IQR
quartiles = data["Income"].quantile([0.25, 0.75])
iqr = quartiles[0.75] - quartiles[0.25]

# Identify Outliers
lower_bound = quartiles[0.25] - 1.5 * iqr
upper_bound = quartiles[0.75] + 1.5 * iqr

# Filtering
income_outliers = data[(data["Income"] < lower_bound) | 
                (data["Income"] > upper_bound)]

print("IQR: ", iqr)
print("outliers: ")
print(income_outliers[["ID", "Year_Old", "Income", "CustomerFor", 
                "Marital_Status", "MntSpentTotal"]]\
                    .sort_values("Income", ascending=False))

# Extracting logically incoherent Outlier from the Dataset
income_excluded = data.drop(2233, inplace=True)

IQR:  33383.5
outliers: 
         ID  Year_Old  Income CustomerFor Marital_Status  MntSpentTotal
2233   9432        47  666666   4059 days       Together             62
617    1503        48  162397   4058 days       Together            107
687    1501        42  160803   4361 days        Married           1717
1300   5336        53  157733   4057 days       Together             59
164    8475        51  157243   3787 days        Married           1608
1653   4931        47  157146   4093 days       Together           1730
2132  11181        75  156924   3971 days        Married              8
655    5555        49  153924   3809 days       Divorced              6


In [19]:
# Storing excluded entries

data_excluded = pd.concat([income_excluded, 
                          Year_Old_outliers, 
                          marital_filt, 
                          row_nan])

print(data_excluded[["ID", "Year_Old", "Income", "CustomerFor", 
                "Marital_Status", "MntSpentTotal"]]\
                    .sort_values("ID"))

         ID  Year_Old  Income CustomerFor Marital_Status  MntSpentTotal
153      92        36   34176   3715 days          Alone             89
131     433        66   61331   4143 days          Alone            632
2177    492        51   48432   4286 days           YOLO            424
339    1150       125   83532   3943 days       Together           1853
133    1295        61    <NA>   3989 days        Married            725
2061   1612        43    <NA>   4061 days         Single             47
10     1994        41    <NA>   3893 days        Married             19
312    2437        35    <NA>   4058 days        Married           1611
319    2863        54    <NA>   3977 days         Single           1052
1382   2902        66    <NA>   4331 days       Together             45
2081   3117        69    <NA>   3921 days         Single            450
1386   3769        52    <NA>   3786 days       Together             42
1383   4345        60    <NA>   3835 days         Single        

## New Categorical columns

In [20]:
# Age binning categories

print(data["Year_Old"].describe()[["min", "mean", "max"]])

bins = [25, 35, 45, 55, 65, 75, 2000]
labels = ["25_34", "35_44", "45_54", "55_64", "65_74", "75_above"]

data["Age_cat"] = pd.cut(data["Year_Old"], bins, labels=labels, right=False)

print("\n", data[["ID", "Year_Old", "Age_cat", "Income"]].sample(5), sep='')

min     28.000000
mean    55.101134
max     84.000000
Name: Year_Old, dtype: float64

         ID  Year_Old   Age_cat  Income
286   10983        72     65_74   75278
600   10277        63     55_64   30081
2153   7699        55     55_64   40590
846   10343        33     25_34   61618
1487   5529        78  75_above   64014


In [21]:
# Income binning categories

print(data["Income"].describe()[["min", "mean", "max"]])

labels = [f"D{i+1}" for i in np.arange(0,10)]

data["Income_cat"] = pd.cut(data["Income"], 10, precision=0, labels=labels)

print("\n", data[["ID", "Age_cat", "Income", "Income_cat"]].sample(5), sep='')


min          1730.0
mean    51954.61542
max        162397.0
Name: Income, dtype: Float64

         ID Age_cat  Income Income_cat
1325   9212   55_64   72828         D5
1792   7960   55_64   82122         D6
479   10581   45_54   41124         D3
1034   1020   45_54   79593         D5
912    9760   55_64   52278         D4


In [22]:
# Total amount spent binning categories

print(data["MntSpentTotal"].describe()[["min", "mean", "max"]])

data["MntTotal_cat"], intervals = pd.cut(data["MntSpentTotal"], 6, 
                                         precision=0, retbins=True)

# Exploring bins:
# print("\n", data["MntTotal_cat"].value_counts().sort_index(), sep='')

# Creating new, more descriptive bins
temp, first_int = pd.cut(np.arange(2, 426), 5, retbins=True)
bins = list(first_int) + list(intervals[2:])

data["MntTotal_cat"], intervals = pd.cut(data["MntSpentTotal"], bins, 
                                         precision=0, right=False, 
                                         retbins=True)

print("\n", data[["ID", "Age_cat", 
                  "Income", "MntTotal_cat", 
                  "MntSpentTotal"]].sample(5), sep='')


min        5.000000
mean     607.380499
max     2525.000000
Name: MntSpentTotal, dtype: float64

        ID Age_cat  Income      MntTotal_cat  MntSpentTotal
940   8957   65_74   64831   [845.0, 1265.0)           1120
1653  4931   45_54  157146  [1685.0, 2105.0)           1730
2098  1103   45_54   81929  [1685.0, 2105.0)           2053
1092  4944   65_74   71367    [425.0, 845.0)            777
566   9120   55_64   66565   [845.0, 1265.0)           1065


In [23]:
# Recency binning categories

print(data["Recency"].describe()[["min", "mean", "max"]])

labels = ["0_24", "25_49", "50_74", "75_99"]

data["Recency_cat"] = pd.cut(data["Recency"], 4, precision=0, labels=labels)

print("\n", data[["MntTotal_cat", "Age_cat", 
                  "Income_cat", "Recency_cat"]].sample(5), sep='')


min      0.000000
mean    49.082993
max     99.000000
Name: Recency, dtype: float64

          MntTotal_cat Age_cat Income_cat Recency_cat
2032       [2.0, 87.0)   45_54         D1       50_74
141    [845.0, 1265.0)   45_54         D4        0_24
999     [340.0, 425.0)   65_74         D3       25_49
430   [1685.0, 2105.0)   55_64         D6        0_24
2128  [1265.0, 1685.0)   55_64         D4        0_24


In [24]:
# Check duplicated customers
duplicated = data["ID"].duplicated().any()
print(f"There are duplicated customers based on 'ID' column?: {duplicated}")

There are duplicated customers based on 'ID' column?: False


## Saving new Dataset

In [25]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2205 entries, 0 to 2239
Data columns (total 39 columns):
 #   Column               Non-Null Count  Dtype          
---  ------               --------------  -----          
 0   ID                   2205 non-null   int64          
 1   Year_Birth           2205 non-null   int64          
 2   Year_Old             2205 non-null   int64          
 3   CustomerFor          2205 non-null   timedelta64[ns]
 4   Dt_Customer          2205 non-null   datetime64[ns] 
 5   Education            2205 non-null   category       
 6   Marital_Status       2205 non-null   category       
 7   Income               2205 non-null   Int64          
 8   Kidhome              2205 non-null   int64          
 9   Teenhome             2205 non-null   int64          
 10  ChildrenHome         2205 non-null   int64          
 11  Recency              2205 non-null   int64          
 12  MntSpentTotal        2205 non-null   int64          
 13  MntWines             22

In [26]:
# Saving DataFrame as csv
data.to_csv("../data/ifood_cleaned.csv", index=False)

# Saving dtypes of each column
data_dtypes = data.dtypes.to_frame('dtypes').reset_index()
dict = data_dtypes.set_index('index')['dtypes'].astype(str).to_dict()

with open('../data/cleaned_dtypes.json', 'w') as f:
    json.dump(dict, f)

# Storing excluded rows
data.to_csv("../data/ifood_excluded.csv", index=False)

In [27]:
# Notebook info

!pip list

process = psutil.Process()
memory_used = process.memory_info().rss / (1024 ** 2)  # MB

print(f"\n\tMemory used: {memory_used:.2f} MB")


Package                   Version
------------------------- ---------
asttokens                 2.4.1
attrs                     23.2.0
Brotli                    1.1.0
certifi                   2024.7.4
cffi                      1.16.0
charset-normalizer        3.3.2
colorama                  0.4.6
comm                      0.2.2
contourpy                 1.2.1
cycler                    0.12.1
Cython                    3.0.10
debugpy                   1.8.2
decorator                 5.1.1
exceptiongroup            1.2.0
executing                 2.0.1
fastjsonschema            2.20.0
fonttools                 4.51.0
h2                        4.1.0
hpack                     4.0.0
hyperframe                6.0.1
idna                      3.7
importlib_metadata        8.0.0
importlib_resources       6.4.0
iniconfig                 2.0.0
ipykernel                 6.29.5
ipython                   8.26.0
ipywidgets                8.1.3
jedi                      0.19.1
jsonschema              